In [ ]:
### Import de bibliotecas para manipulação dos dados
import pandas as pd
import numpy as np
import math

from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler, StandardScaler, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression

In [ ]:
#Caminho e leitura do Dataframe
df = pd.read_csv('../data/raw/Orange_Quality_Data.csv')

#Análise das 10 primeiras linhas 
df.head(10)

In [ ]:
# Leitura do CSV
df_dicionario_dados = pd.read_csv('../data/external/dicionario_de_dados_laranjas.csv')

#Análise das 10 primeiras linhas 
#df_dicionario_dados.head(10)

In [ ]:
#Checagem de nomes de coluna para facilitar elaboração do código 
#print(df_dicionario_dados.columns)

In [ ]:
#Organização da busca pelo tipo de variável

variavel_alvo = 'Quality'

# Renomeando colunas (opcional)
df_dicionario_dados = df_dicionario_dados.rename(columns={'Variável': 'Variavel', 'Subtipo': 'Subtipo'})


variavel_nominal = (
    df_dicionario_dados
    .query("Subtipo == 'Nominal' and Variavel != @variavel_alvo")
    .Variavel
    .to_list()
)
print(f"Variáveis qualitativo: nominal")
print(variavel_nominal)
print(" ")

variavel_quanti_continua = (
    df_dicionario_dados
    .query("Subtipo == 'Contínua' and Variavel != @variavel_alvo")
    .Variavel #Nome da coluna que tem as variáveis
    .to_list() #Método que lista
)
print(f"Variável quantitativo: Contínua")
print(variavel_quanti_continua)
print(" ")
variavel_quanti_discreta = (
    df_dicionario_dados
    .query("Subtipo == 'Discreto' and Variavel != @variavel_alvo")
    .Variavel #Nome da coluna que tem as variáveis
    .to_list() #Método que lista
)
print(f"Variável quantitativo: Discreto")
print(variavel_quanti_discreta)
print(" ")

#x = df.drop(columns=[variavel_alvo])
#y = df[variavel_alvo]

## Pré-processamento

In [ ]:
processamento_nominal = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='most_frequent')), # tratamento para dados faltantes
    ("encoding", OneHotEncoder(sparse_output=False)), # codificação de variáveis
])

processamento_continuos = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='mean')), #Tratamento de dados faltantes com substituição pela MÉDIA
    ("normalization", StandardScaler()), #Normalização de variáveis
     
])
processamento_discreto = Pipeline(steps=[
    ("missing", KNNImputer()), #Tratamento de dados faltantes com substituição pela MEDIANA
    ("normalization", StandardScaler()), #Normalização de variáveis
     
])

# Combinar transformações e aplicar 

In [ ]:
pre_processador = ColumnTransformer([
    ("Contínua", processamento_continuos, variavel_quanti_continua),
    ("Discreto", processamento_discreto, variavel_quanti_discreta),
    ("Nominal", processamento_nominal, variavel_nominal)
])

df_preprocessed = pre_processador.fit_transform(df)

df_preprocessed

## Método de validação cruzada

In [ ]:
#Rever e corrir se necessário

#k-fold
# Create a Pipeline
model = Pipeline([
    ('encoder', OneHotEncoder(handle_unknown='ignore')),
    ('scaler', StandardScaler(with_mean=False)),  # Optional scaler for numerical features
    ('regressor', LinearRegression())
])

# Perform cross-validation
scores = cross_val_score(model, df, y, cv=10, scoring='neg_mean_squared_error')

print(scores)

mean_score = np.mean(scores)
std_score = np.std(scores)

print("Média:", mean_score)
print("Desvio Padrão:", std_score)